In [40]:
#importing required pakages for text processing 
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from stop_words import get_stop_words
from nltk.stem.snowball import SnowballStemmer
from gensim import corpora, models
import gensim

In [41]:
import nltk
#nltk.download("stopwords")
import pandas as pd
import json
from pandas.io.json import json_normalize
#import ijson

In [42]:
#loading a single json file
data = pd.read_json('dataset/Musical_Instruments_5.json', lines = True)
data1= pd.DataFrame.from_dict(data, orient='columns')
len(data1)


10261

In [43]:
#removing the stop words
nltk_stpwd = stopwords.words('english')
stop_words_stpwd = get_stop_words('en')
merged_stopwords = list(set(nltk_stpwd + stop_words_stpwd))

#print(len(set(merged_stopwords)))
#print(merged_stopwords[:10])

In [44]:
sb_stemmer = SnowballStemmer('english')

In [45]:
num_reviews = data1.shape[0]


In [46]:
doc_set = [data1.reviewText[i] for i in range(num_reviews)]
#doc_set

In [47]:
%%time
#time taken to tokenize on a single core
texts = []

for doc in doc_set:
    # putting our three steps together
    tokens = tokenizer.tokenize(doc.lower())
    stopped_tokens = [token for token in tokens if not token in merged_stopwords]
    stemmed_tokens = [sb_stemmer.stem(token) for token in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)


CPU times: user 22.9 s, sys: 2.02 s, total: 24.9 s
Wall time: 24.7 s


In [48]:
def tokenize_and_pos_tag(doc):
    tokens = tokenizer.tokenize(doc.lower())
    stopped_tokens = [token for token in tokens if not token in merged_stopwords]
    stemmed_tokens = [sb_stemmer.stem(token) for token in stopped_tokens]
    return stemmed_tokens



In [49]:
%%time
#time taken on 
from multiprocessing import Pool
pool = Pool(processes=6)
tokens = pool.map(tokenize_and_pos_tag, doc_set)

CPU times: user 220 ms, sys: 180 ms, total: 400 ms
Wall time: 5.48 s


In [51]:
from collections import defaultdict
frequency = defaultdict(int)
for text in tokens:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1]
          for text in tokens]



In [52]:
# Gensim's Dictionary encapsulates the mapping between normalized words and their integer ids.
texts_dict = corpora.Dictionary(texts)
#texts_dict.save('auto_review.dict') # lets save to disk for later use
# Examine each token’s unique id
#print(texts_dict)
#print(texts_dict.token2id)

In [53]:
#import operator
#texts_dict.filter_extremes(no_below=30, no_above=0.15) # inlace filter#
#print(texts_dict)
#print("top terms:")
#print(sorted(texts_dict.token2id.items(), key=operator.itemgetter(1), reverse = False)[:10])

In [54]:
corpus = [texts_dict.doc2bow(text) for text in texts]
len(corpus)


10261

In [55]:
## Topic Modeling on single core
import time
start_time = time.time()
lda_model = gensim.models.LdaModel(corpus,  num_topics=5, id2word=texts_dict, passes=20)
print(time.time() - start_time)
## the time is shown in seconds

482.800997019


In [56]:
#model for 5 words and 5 topics
lda_model.show_topics(num_words=5, num_topics=5)

[(0,
  u'0.024*"pedal" + 0.021*"sound" + 0.019*"amp" + 0.014*"use" + 0.011*"like"'),
 (1,
  u'0.059*"string" + 0.029*"guitar" + 0.023*"sound" + 0.019*"play" + 0.012*"tune"'),
 (2,
  u'0.022*"guitar" + 0.015*"work" + 0.015*"one" + 0.014*"strap" + 0.013*"good"'),
 (3,
  u'0.022*"tuner" + 0.019*"use" + 0.017*"record" + 0.011*"one" + 0.009*"tune"'),
 (4,
  u'0.026*"pick" + 0.022*"stand" + 0.022*"capo" + 0.021*"use" + 0.020*"guitar"')]

In [57]:
%%time
## Multicore Topic Modeling with time taken in minutes
#import time
#start_time = time.time()
lda = gensim.models.LdaMulticore(corpus, id2word=texts_dict, num_topics=5, passes=20, workers=5)
#print(time.time() - start_time)

CPU times: user 3min 27s, sys: 19.2 s, total: 3min 47s
Wall time: 3min 28s


In [58]:
#lda model for 5 words and 5 topics 
lda.show_topics(num_words=5, num_topics=5)

[(0,
  u'0.018*"use" + 0.014*"record" + 0.007*"work" + 0.007*"get" + 0.006*"one"'),
 (1,
  u'0.025*"guitar" + 0.015*"one" + 0.014*"use" + 0.011*"work" + 0.010*"well"'),
 (2,
  u'0.049*"string" + 0.020*"guitar" + 0.018*"use" + 0.016*"tune" + 0.016*"tuner"'),
 (3,
  u'0.023*"sound" + 0.017*"use" + 0.017*"mic" + 0.015*"amp" + 0.010*"record"'),
 (4,
  u'0.032*"pedal" + 0.017*"sound" + 0.014*"use" + 0.013*"amp" + 0.012*"like"')]